In [2]:
import yaml
import json

# Load files
with open("config_a.yml") as f:
    config = yaml.safe_load(f)

with open("Resonances.yml") as f:
    resonances = yaml.safe_load(f)

with open("final_params.json") as f:
    final_params = json.load(f)["value"]

# Kinematics
kinematics = {
    "initial_state": {
        "index": 0,
        "name": "Bp",
        "mass": config["particle"]["$top"]["Bp"]["mass"],
        "spin": str(config["particle"]["$top"]["Bp"]["J"])
    },
    "final_state": [
        {
            "index": 1,
            "name": "Dst",
            "mass": config["particle"]["Dst"]["mass"],
            "spin": str(config["particle"]["Dst"]["J"])
        },
        {
            "index": 2,
            "name": "D",
            "mass": config["particle"]["$finals"]["D"]["mass"],
            "spin": str(config["particle"]["$finals"]["D"]["J"])
        },
        {
            "index": 3,
            "name": "K",
            "mass": config["particle"]["$finals"]["K"]["mass"],
            "spin": str(config["particle"]["$finals"]["K"]["J"])
        }
    ]
}

# Chains from config_a and Resonances
chains = []
functions = []

for comb in ["DstD", "DstK", "DK"]:
    for res_name in config["particle"][comb]:

        res_name_res = res_name
        if res_name in ["X0(2900)","X1(2900)"]:
            res_name_res = ["XDK","XDK2"][config["particle"]["DK"].index(res_name)]

        res_data = resonances[res_name_res]

        mass = final_params.get(f"{res_name_res}_mass", res_data.get("mass", res_data.get("m0", None)))
        width = final_params.get(f"{res_name_res}_width", res_data.get("width", res_data.get("g0", None)))

        if comb == "DstD":
            weight_real = final_params.get(f"Bp->{res_name}.K{res_name}->Dst.DDst->D0.pi_total_0r", 1.0)
            weight_imag = final_params.get(f"Bp->{res_name}.K{res_name}->Dst.DDst->D0.pi_total_0i", 0.0)
            weight = f"{weight_real} + {weight_imag} i"

        if comb == "DK":
            weight_real = final_params.get(f"Bp->{res_name}.Dst{res_name}->D.KDst->D0.pi_total_0r", 1.0)
            weight_imag = final_params.get(f"Bp->{res_name}.Dst{res_name}->D.KDst->D0.pi_total_0i", 0.0)
            weight = f"{weight_real} + {weight_imag} i"

        # Function-Definition
        func_name = f"BW_{res_name}"
        functions.append({
            "name": func_name,
            "type": "BreitWigner",
            "mass": str(mass),
            "width": str(width)
        })

        # Chain-Definition
        if comb == "DstD":
            chains.append({
                "name": res_name,
                "topology": [[1, 2], 3],  # DstD + K
                "propagators": [{
                    "name": res_name,
                    "node": [1, 2],
                    "spin": str(res_data["J"]),
                    "parametrization": func_name
                }],
                "vertices": [{
                    "type": "helicity",
                    "node": [0, [1, 2]],
                    "helicities": []
                }],
                "weight": weight
            })

        if comb == "DK":
            chains.append({
                "name": res_name,
                "topology": [[2, 3], 1],  # DK + Dst
                "propagators": [{
                    "name": res_name,
                    "node": [2, 3],
                    "spin": str(res_data["J"]),
                    "parametrization": func_name
                }],
                "vertices": [{
                    "type": "helicity",
                    "node": [0, [2, 3]],
                    "helicities": []
                }],
                "weight": weight
            })

# JSON building
amp_json = {
    "kinematics": kinematics,
    "reference_topology": [[1, 2], 3],
    "chains": chains,
    "functions": functions
}

with open("amplitude_serialization.json", "w") as f:
    json.dump(amp_json, f, indent=2)


In [3]:
all_res = config["particle"]["DK"]
DKs = ["XDK", "XDK2"]

print(["XDK","XDK2"][config["particle"]["DK"].index("X1(2900)")])

#print(final_params.get(f"Bp->{res_name}.Dst{res_name}->D.KDst->D0.pi_total_0r", 1.0))
#print(final_params.get(f"Bp->{res_name}.Dst{res_name}->D.KDst->D0.pi_total_0i", 0.0))
#print(res_name in resonances)

XDK2
